# Notebook to convert all meshes from Acronym dataset into voxel grids and save them as .npy files

Second half of the notebook has the functionality to save voxel grids as .h5 files

In [36]:
import os
import json
import h5py
import trimesh
import trimesh.path
import trimesh.transformations as tra
import numpy as np
from acronym_tools import load_mesh, load_grasps, create_gripper_marker

import os, json
import trimesh
import mesh2sdf
import numpy as np
from scipy.spatial.transform import Rotation as R

def create_voxel_grid(mesh, n=32, scale=1):
    """
    Uses the the mesh2sdf library to convert the mesh into a (n x n x n) voxel grid
    The values within the grid are sdf (signed distance function) values
    Input - 
        1. mesh_path --> Path to mesh file (.obj file)
        2. n --> size of voxel grid
    Output - 
        1. mesh --> mesh object after loading, normalizing and fixing mesh
        2. sdf --> (n x n x n) numpy array 
    """

    # Load mesh file
    # mesh = trimesh.load(mesh_path, force='mesh')

    mesh_scale = scale
    size = n
    level = 2 / size

    # normalize mesh
    vertices = mesh.vertices
    bbmin = vertices.min(0)
    bbmax = vertices.max(0)
    center = (bbmin + bbmax) * 0.5
    scale = 2.0 * mesh_scale / (bbmax - bbmin).max()
    vertices = (vertices - center) * scale

    sdf, mesh = mesh2sdf.compute(
        vertices, mesh.faces, size, fix=True, level=level, return_mesh=True)

    # output
    mesh.vertices = mesh.vertices / scale + center

    return mesh, sdf

def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)

grasp_dir = "/home/username/data/grasps"
grasp_paths = [os.path.join(grasp_dir, i) for i in os.listdir(grasp_dir)]
print(len(grasp_paths))

8836


In [37]:
grasp_path = grasp_paths[0]
mesh_root = "/home/username/data"

data = h5py.File(grasp_path, "r")
mesh_fname = data["object/file"][()].decode('utf-8')
mesh_scale = data["object/scale"][()]

obj_mesh = trimesh.load(os.path.join(mesh_root, mesh_fname))
obj_mesh = obj_mesh.apply_scale(mesh_scale)


### Looping over all meshes and converting them into .npy files

In [38]:
from tqdm import tqdm

voxel_dirname = "/home/username/data/voxel_grids"
for i in tqdm(range(len(grasp_paths))):
    grasp_path = grasp_paths[i]
    mesh_root = "/home/username/data"

    data = h5py.File(grasp_path, "r")
    mesh_fname = data["object/file"][()].decode('utf-8')
    mesh_scale = data["object/scale"][()]

    temp = mesh_fname.split("/")
    converted_dirname = os.path.join(voxel_dirname, temp[1])
    makedirs(converted_dirname)
    output_fname = os.path.splitext(temp[2])[0] + ".npy"
    output_fname = os.path.join(converted_dirname, output_fname)
    # print(output_fname)

    obj_mesh = trimesh.load(os.path.join(mesh_root, mesh_fname),  file_type='obj', force='mesh')
    mesh, sdf = create_voxel_grid(obj_mesh, scale=mesh_scale)
    # print(sdf.shape)

    np.save(output_fname, sdf)
    # break;

 38%|███▊      | 3368/8836 [23:23<31:12,  2.92it/s]  /opt/conda/lib/python3.10/site-packages/trimesh/grouping.py:99: RuntimeWarning: invalid value encountered in cast
  stacked = np.column_stack(stacked).round().astype(np.int64)
100%|██████████| 8836/8836 [1:06:09<00:00,  2.23it/s]
